<center><img src="https://assets.datacamp.com/production/repositories/6005/datasets/0c64652120b80cfe2762012d7252b439138be223/piggy_bank.jpg" alt="Piggy bank" width="800" height="300"></center>

<br>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. It is recommended to use `pandas` for these tasks.

Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll provide the database design script as a `.sql` file that they can then run. 

You have been asked to design a database that will have three tables:

## client

| column | data type | description |
|--------|-----------|-------------|
| `id` | `serial` | Client ID - primary key |
| `age` | `integer` | Client's age in years |
| `job` | `text` | Client's type of job |
| `marital` | `text` | Client's marital status |
| `education` | `text` | Client's level of education |
| `credit_default` | `boolean` | Whether the client's credit is in default |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) |
| `loan` | `boolean` | Whether the client has an existing personal loan |

<br>

## campaign

| column | data type | description |
|--------|-----------|-------------|
| `campaign_id` | `serial` | Campaign ID - primary key |
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign |
| `contact_duration` | `integer` | Last contact duration in seconds |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign |
| `previous_outcome` | `boolean` | Outcome of the previous campaign |
| `campaign_outcome` | `boolean` | Outcome of the current campaign |
| `last_contact_date` | `date` | Last date the client was contacted |

<br>

## economics

| column | data type | description |
|--------|-----------|-------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) |
| `number_employed` | `float` | Number of employees (quarterly indicator)| 

In [83]:
import pandas as pd
import numpy as np

In [84]:
client = pd.DataFrame()
campaign = pd.DataFrame()
economics = pd.DataFrame()

In [85]:
client['id'] = df['client_id']
client['age'] = df['age']
client['job'] = df['job']
client['marital'] = df['marital']
client['education'] = df['education']
client['credit_default'] = df['credit_default']
client['housing'] = df['housing']
client['loan'] = df['loan']

In [86]:
client['education'] = client['education'].str.replace('.','_')
client['education'] = client['education'].replace('unknown',np.NaN)
client['job'] = client['job'].str.replace('.','')

In [87]:
client.head(2)

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic_4y,no,no,no
1,1,57,services,married,high_school,unknown,no,no


In [88]:
campaign['client_id'] = df['client_id']
campaign['campaign_id'] = 1
campaign['number_contacts'] = df['campaign']
campaign['contact_duration'] = df['duration']
campaign['pdays'] = df['pdays']
campaign['previous_campaign_contacts'] = df['previous']
campaign['previous_outcome'] = df['poutcome']
campaign['campaign_outcome'] = df['y']
campaign['last_contact_date'] = pd.to_datetime('2022'+'-'+df['month']+'-'+df['day_of_week'],format="%Y-%b-%a")

In [89]:
campaign['previous_outcome'] = campaign['previous_outcome'].str.replace('success','1')
campaign['previous_outcome'] = campaign['previous_outcome'].str.replace('failure','0')
campaign['previous_outcome'] = campaign['previous_outcome'].replace('nonexistent',np.NaN)
campaign['campaign_outcome'] = campaign['campaign_outcome'].str.replace('yes','1')
campaign['campaign_outcome'] = campaign['campaign_outcome'].str.replace('no','0')
campaign = campaign.iloc[:,[1,0,2,3,4,5,6,7,8]]

In [90]:
campaign.head(10)

,campaign_id,client_id,number_contacts,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
0,1,0,1,261,999,0,NaN,0,2022-05-01
1,1,1,1,149,999,0,NaN,0,2022-05-01
2,1,2,1,226,999,0,NaN,0,2022-05-01
3,1,3,1,151,999,0,NaN,0,2022-05-01
4,1,4,1,307,999,0,NaN,0,2022-05-01
5,1,5,1,198,999,0,NaN,0,2022-05-01
6,1,6,1,139,999,0,NaN,0,2022-05-01
7,1,7,1,217,999,0,NaN,0,2022-05-01
8,1,8,1,380,999,0,NaN,0,2022-05-01
9,1,9,1,50,999,0,NaN,0,2022-05-01


In [91]:
economics['client_id'] = df['client_id']
economics['emp_var_rate'] = df['emp_var_rate']
economics['cons_price_idx'] = df['cons_price_idx']
economics['euribor_three_months'] = df['euribor3m']
economics['number_employed'] = df['nr_employed']

In [92]:
client.to_csv("client.csv", index=False)
campaign.to_csv("campaign.csv", index=False)
economics.to_csv("economics.csv", index=False)

# Store and print database_design
client_table = """CREATE TABLE client
(
    id SERIAL PRIMARY KEY,
    age INTEGER,
    job TEXT,
    marital TEXT,
    education TEXT,
    credit_default BOOLEAN,
    housing BOOLEAN,
    loan BOOLEAN
);
\copy client from 'client.csv' DELIMITER ',' CSV HEADER
"""

campaign_table = """CREATE TABLE campaign
(
    campaign_id SERIAL PRIMARY KEY,
    client_id SERIAL references client (id),
    number_contacts INTEGER,
    contact_duration INTEGER,
    pdays INTEGER,
    previous_campaign_contacts INTEGER,
    previous_outcome BOOLEAN,
    campaign_outcome BOOLEAN,
    last_contact_date DATE    
);
\copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER
"""

economics_table = """CREATE TABLE economics
(
    client_id INTEGER references client (id),
    emp_var_rate FLOAT,
    cons_price_idx FLOAT,
    euribor_three_months FLOAT,
    number_employed FLOAT
);
\copy economics from 'economics.csv' DELIMITER ',' CSV HEADER
"""

In [93]:
df = pd.read_csv('bank_marketing.csv')
display(df.head(), df.info(), df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   client_id       41188 non-null  int64  
 1   age             41188 non-null  int64  
 2   job             41188 non-null  object 
 3   marital         41188 non-null  object 
 4   education       41188 non-null  object 
 5   credit_default  41188 non-null  object 
 6   housing         41188 non-null  object 
 7   loan            41188 non-null  object 
 8   contact         41188 non-null  object 
 9   month           41188 non-null  object 
 10  day_of_week     41188 non-null  object 
 11  duration        41188 non-null  int64  
 12  campaign        41188 non-null  int64  
 13  pdays           41188 non-null  int64  
 14  previous        41188 non-null  int64  
 15  poutcome        41188 non-null  object 
 16  emp_var_rate    41188 non-null  float64
 17  cons_price_idx  41188 non-null 

,client_id,age,job,marital,education,credit_default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


None

,client_id,age,duration,campaign,pdays,previous,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed
count,41188.00000,41188.00000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000
mean,20593.50000,40.02406,258.285010,2.567593,962.475454,0.172963,0.081886,93.575664,-40.502600,3.621291,5167.035911
std,11890.09578,10.42125,259.279249,2.770014,186.910907,0.494901,1.570960,0.578840,4.628198,1.734447,72.251528
min,0.00000,17.00000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,10296.75000,32.00000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,20593.50000,38.00000,180.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,30890.25000,47.00000,319.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,41187.00000,98.00000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000
